# Language modelling

In [8]:
!pip install transformers
!pip install sacremoses

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 880 kB 4.0 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895259 sha256=dbe878f9f07f517ecfa089d9a4b0a9490dc47533c85996dc9be75fdc5dd858b3
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses


In [10]:
from transformers import pipeline
import torch

Download three Polish models from the Huggingface repository.

## HerBERT 

In [9]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer_herbert = AutoTokenizer.from_pretrained("allegro/herbert-base-cased")
herbert = AutoModelForMaskedLM.from_pretrained("allegro/herbert-base-cased")

Downloading:   0%|          | 0.00/654M [00:00<?, ?B/s]

Some weights of the model checkpoint at allegro/herbert-base-cased were not used when initializing BertForMaskedLM: ['cls.sso.sso_relationship.bias', 'cls.sso.sso_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## Polbert - Polish BERT

In [5]:
from transformers import BertForMaskedLM, BertTokenizer

tokenizer_bert = BertTokenizer.from_pretrained("dkleczek/bert-base-polish-uncased-v1")
bert = BertForMaskedLM.from_pretrained("dkleczek/bert-base-polish-uncased-v1")

Downloading:   0%|          | 0.00/478 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/531M [00:00<?, ?B/s]

Some weights of the model checkpoint at dkleczek/bert-base-polish-uncased-v1 were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/495k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

## Bert base multilingual

In [11]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer_bert_bg = AutoTokenizer.from_pretrained("DeepPavlov/bert-base-bg-cs-pl-ru-cased")
bert_bg = AutoModelForMaskedLM.from_pretrained("DeepPavlov/bert-base-bg-cs-pl-ru-cased")

Some weights of the model checkpoint at DeepPavlov/bert-base-bg-cs-pl-ru-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [12]:
models = [
    {'name':"HerBERT", 'model': herbert,'tokenizer': tokenizer_herbert},
    {'name':"Polish BERT", 'model':bert,'tokenizer': tokenizer_bert},
    {'name':"Bert base multinigual", 'model':bert_bg,'tokenizer':tokenizer_bert_bg}
]

In [14]:
def predict_word(sentence, model, tokenizer):
  sentence = sentence.replace('[MASK]', tokenizer.mask_token)
  inputs = tokenizer(sentence, return_tensors="pt")
  mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]
  token_logits = model(**inputs).logits
  mask_token_logits = token_logits[0, mask_token_index, :]
  for token in torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist():
    print(sentence.replace(tokenizer.mask_token, tokenizer.decode([token])))
  print()

Devise a method to test if the langage model understands Polish cases. E.g. testing for nominal case could be expressed as "Warszawa to największe [MASK]", and the masked word should be in nominative case. Create sentences for each case.

In [15]:
sentences = [
    ("Mianownik", "Warszawa to największe [MASK]."),
    ("Dopełniacz", "Mateusz wyszedł z domu zapominając [MASK]."),
    ("Celownik", "Tata podarował prezent [MASK]."),
    ("Biernik", "Po wpinaczce górskiej możemy podziwiać [MASK]."),
    ("Narzędnik", "Basia idzie na spacer z [MASK]."),
    ("Miejscownik", "Na ostatnim spotkaniu w pracy rozmiawialiśmy o [MASK]."),
    ("Wołacz", "Witaj [MASK]!")
]


In [16]:
for model in models:
  print('-'*100)
  print(model['name'])
  for case, sentence in sentences:
    print(case)
    predict_word(sentence, model['model'], model['tokenizer'])

----------------------------------------------------------------------------------------------------
HerBERT
Mianownik
Warszawa to największe miasto.
Warszawa to największe lotnisko.
Warszawa to największe centrum.
Warszawa to największe miasta.
Warszawa to największe atrakcje.

Dopełniacz
Mateusz wyszedł z domu zapominając rodziców.
Mateusz wyszedł z domu zapominając telefon.
Mateusz wyszedł z domu zapominając o.
Mateusz wyszedł z domu zapominając go.
Mateusz wyszedł z domu zapominając się.

Celownik
Tata podarował prezent dzieciom.
Tata podarował prezent mamie.
Tata podarował prezent synowi.
Tata podarował prezent babci.
Tata podarował prezent dziecku.

Biernik
Po wpinaczce górskiej możemy podziwiać widoki.
Po wpinaczce górskiej możemy podziwiać Tatry.
Po wpinaczce górskiej możemy podziwiać góry.
Po wpinaczce górskiej możemy podziwiać ..
Po wpinaczce górskiej możemy podziwiać krajobraz.

Narzędnik
Basia idzie na spacer z psem.
Basia idzie na spacer z mamą.
Basia idzie na spacer z rod

Devise a method to test long-range relationships such as gender. E.e. you can use two verbs where withe masculine and feminine gender, where one of the verbs is masked. Both verbs should have the same gender, assuming the subject is the same. Define at least 3 such sentences.

In [17]:
long_range_sentences = [
    "Ala poszła do parku, gdzie [MASK] Tomka.",
    "Robert poleciał na Alaske, gdzie [MASK] jelenia.",
    "Byliśmy w kinie i [MASK] film."
]

In [19]:
for model in models:
  print('-'*100)
  print(model['name'])
  for sentence in long_range_sentences:
    predict_word(sentence, model['model'], model['tokenizer'])

----------------------------------------------------------------------------------------------------
HerBERT
Ala poszła do parku, gdzie poznała Tomka.
Ala poszła do parku, gdzie spotkała Tomka.
Ala poszła do parku, gdzie zobaczyła Tomka.
Ala poszła do parku, gdzie widziała Tomka.
Ala poszła do parku, gdzie znalazła Tomka.

Robert poleciał na Alaske, gdzie spotkał jelenia.
Robert poleciał na Alaske, gdzie zobaczył jelenia.
Robert poleciał na Alaske, gdzie złapał jelenia.
Robert poleciał na Alaske, gdzie zaatakował jelenia.
Robert poleciał na Alaske, gdzie znalazł jelenia.

Byliśmy w kinie i obejrzeli film.
Byliśmy w kinie i widzieliśmy film.
Byliśmy w kinie i zrobiliśmy film.
Byliśmy w kinie i był film.
Byliśmy w kinie i dostaliśmy film.

----------------------------------------------------------------------------------------------------
Polish BERT
Ala poszła do parku, gdzie poznała Tomka.
Ala poszła do parku, gdzie spotkała Tomka.
Ala poszła do parku, gdzie zabito Tomka.
Ala poszła do

Check if the model captures real-world knolwedge. For instance a sentence "[MASK] wrze w temperaturze 100 stopni, a zamarza w temperaturze 0 stopni Celsjusza." checks if the model "knows" the description of water. Define at least 3 such sentences.

In [20]:
real_world_sentences = [
    "[MASK] wrze w temperaturze 100 stopni, a zamarza w temperaturze 0 stopni Celsjusza.",
    "Madryt jest stolicą [MASK].",
    "W Polsce mówi sie po [MASK].",
    "Krzysztof Kolumb odkrył [MASK].",
]

In [21]:
for model in models:
  print('-'*100)
  print(model['name'])
  for sentence in real_world_sentences:
    predict_word(sentence, model['model'], model['tokenizer'])

----------------------------------------------------------------------------------------------------
HerBERT
Woda wrze w temperaturze 100 stopni, a zamarza w temperaturze 0 stopni Celsjusza.
Słońce wrze w temperaturze 100 stopni, a zamarza w temperaturze 0 stopni Celsjusza.
Ziemia wrze w temperaturze 100 stopni, a zamarza w temperaturze 0 stopni Celsjusza.
Następnie wrze w temperaturze 100 stopni, a zamarza w temperaturze 0 stopni Celsjusza.
Ciało wrze w temperaturze 100 stopni, a zamarza w temperaturze 0 stopni Celsjusza.

Madryt jest stolicą Hiszpanii.
Madryt jest stolicą Portugalii.
Madryt jest stolicą Włoch.
Madryt jest stolicą Europy.
Madryt jest stolicą kraju.

W Polsce mówi sie po angielsku.
W Polsce mówi sie po polsku.
W Polsce mówi sie po niemiecku.
W Polsce mówi sie po angielskim.
W Polsce mówi sie po niemieckim.

Krzysztof Kolumb odkrył świat.
Krzysztof Kolumb odkrył kosmos.
Krzysztof Kolumb odkrył Kopernika.
Krzysztof Kolumb odkrył złoto.
Krzysztof Kolumb odkrył Księżyc.

-

Answer the following questions:
* Which of the models produced the best results?

In my opinion HerBERT gives the best results.

* Was any of the models able to capture Polish grammar?

HerBERT was the most successful in capturing Polish grammar, it did well in Polish declention, like other models but was the best in capturing correct verb forms.

* Was any of the models able to capture long-distant relationships between the words?

HerBERT was the best model for long-distant relationships, the words made sense and the model was the only that was able to recognize the plural form of Polish verbs.

* Was any of the models able to capture world knowledge?

Bert base multinigual is the best model for real-world knowledge, for the first sentece most of the words are connected with water, the only model that tells in Poland we speak Polish. 
None of the models knows that Christopher Columbus discovered America.

* What are the most striking errors made by the models?

Models in general don't recognize plural form in long-range relationships and sometimes words come out of the blue such as 'Krzysztof Kolumb odkrył ok.', 'Mateusz wyszedł z domu zapominając o.', 'Madryt jest stolicą Madryt.'